In [1]:
#%matplotlib qt5
#%matplotlib inline
#para hacer el grafico animado
%matplotlib notebook  

import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bokeh.plotting as bkplt
import bokeh.models as bkmdls
import bokeh.io as bkio

import matplotlib.cm as cm

from sklearn import cluster,datasets
from mpl_toolkits.mplot3d import Axes3D


from itertools import cycle
from cycler import cycler
import time
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import Birch
from sklearn import mixture

import matplotlib.colors as colors
import seaborn as sns; sns.set()

import time

In [2]:
rutaxls = 'C:/Users/mungu/Documents/DatosWTG.xlsx'
rutaxlsCPot = 'C:/Users/mungu/Dropbox/Doctorado/Python/aero/Curva de potencia vestas 90.xlsx'
#rutaxls = 'C:/Users/mungu/Documents/DatosWTG.xlsx'
#rutaxlsCPot = 'Curva de potencia vestas 90.xlsx'

dataVPxls = pd.read_excel(rutaxls,usecols=[0,1,2],index_col=0,names=['PCTimeStamp','vViento','Pacw'])
dfCPot = pd.read_excel(rutaxlsCPot,usecols=[0,2],index_col=0,names=['vel','pw'])

#marcando los datos faltantes asignando un nan a la fila completa
datamk = dataVPxls
datamk.loc[datamk.isnull().any(axis=1), :] = np.nan

#eliminando filas con NaN. Si busco la fecha anterior debe aparecer error.
cleanData = datamk.dropna()

###calculando la potencia
#la pontencia del archivo de excel es la densidad de potencia acumulada.
#Se tiene que hacer la resta de la potencia siguiente a la anterio y multiplicar por pi*r^2
#el radio es 45m
#si se hace la resta con un array de numpy (values) si se puede restar 
#pues se hace elemento por elemento
dataVP = cleanData.drop('Pacw',axis=1)
dataVP['Pw']= (cleanData.iloc[1:,1].values-cleanData.iloc[0:-1,1]) * np.pi*45**2

#eliminar el ultimo valor pues es NaN
dataVP.drop(dataVP.tail(1).index,inplace=True) 

#Eliminando outliers
dataVP.drop([pd.to_datetime('2016-03-07 09:50:00')],inplace=True)

#leyendo la direccion del viento
dataDirVxls = pd.read_excel(rutaxls,sheet_name=1, usecols=[0,1],index_col=0,names=['PCTimeStamp','Dir'])
#limpiando datos
#marcando los datos faltantes asignando un nan a la fila completa
datamkdir = dataDirVxls
datamkdir.loc[datamkdir.isnull().any(axis=1), :] = np.nan
#eliminando filas con NaN. Si busco la fecha anterior debe aparecer error.
#tambien elimino el ultimo valor como lo hice en los datos de v y p
#el copy es para que no me de la copy warning
dataDir = datamkdir.dropna().copy()
dataDir.drop(dataDir.tail(1).index,inplace=True) 
#eliminando del outlier de viento misma fecha que el de la pontencia
dataDir.drop([pd.to_datetime('2016-03-07 09:50:00')],inplace=True)

direcvrad= np.deg2rad(dataDir['Dir'].values)
velocidades = dataVP.iloc[:]['vViento'].values
vecVel = [-np.sin(direcvrad)*velocidades,np.cos(direcvrad)*velocidades]
vecVelnp=np.array(vecVel).transpose()
#original sin timestamp
#dfVecVel = pd.DataFrame(data=vecVelnp,columns=['vx','vy']
#con timestamp
dfVecVel = pd.DataFrame(data=vecVelnp,columns=['vx','vy'],index=dataVP.index)

In [3]:
n_clusters= 25
#Birch
# forzar n  clusters|
model_birch = Birch(threshold=1.7, n_clusters=n_clusters)
model_birch.fit(dfVecVel.values) 
bch_labels = model_birch.labels_
centroids = model_birch.subcluster_centers_#los centroides siguen siendo 46
n_clusters = np.unique(bch_labels).size

In [4]:
dataDir

,Dir
PCTimeStamp,
2016-01-01 00:00:00,29.6
2016-01-01 00:10:00,31.3
2016-01-01 00:20:00,25.9
2016-01-01 00:30:00,23.8
2016-01-01 00:40:00,24.4
...,...
2017-01-01 23:00:00,9.4
2017-01-01 23:10:00,15.2
2017-01-01 23:20:00,24.9


In [5]:
#orden de aparicion de los clusters kmeans y gm

#ordenar el orden de aparicion segun la magnitud de la vv
bchmagni = np.zeros(n_clusters)
for i in range(n_clusters):
    bchvx = dfVecVel.vx.values[bch_labels==i]
    bchvy = dfVecVel.vy.values[bch_labels==i]
    bchmagni[i]= round(np.mean(np.sqrt(bchvx**2 + bchvy**2)),1) #magnitud de la vv
    

bchord = bchmagni.argsort()


In [6]:
#matriz de potencias
#nombre de las columnas del dataframe
columnas = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25']
dfclpw = pd.DataFrame()
for i in range(n_clusters):
    dfclpw = pd.concat([dfclpw,dataVP.Pw[bch_labels==bchord[i]]], ignore_index=True, axis=1)
dfclpw.columns=columnas[0:n_clusters]
dfclpw.head(20)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25
2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.953050e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.067561e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.048475e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.080284e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.073922e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 00:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.099369e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.978497e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.825815e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.724028e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.704942e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#cambiando los valores de potencia a unos
dfclpw.fillna(0,inplace=True)
dfclpw[dfclpw!=0]=1
dfclpw.head(20)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25
2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#np.nonzero(dfclpw.iloc[10,:])[0][0] deprecated
dfclpw.iloc[10,:].to_numpy().nonzero()[0][0]#actual

14

In [9]:
#matriz de transicion del tamaño del numero de  clusters
mattra = np.zeros([n_clusters,n_clusters])
for row in range(len(dfclpw)-1):
    if dfclpw.iloc[row,:].to_numpy().nonzero()[0].shape[0] >0 and dfclpw.iloc[row+1,:].to_numpy().nonzero()[0].shape[0] >0:
    #if np.nonzero(dfclpw.iloc[row,:])[0].shape[0] >0 and np.nonzero(dfclpw.iloc[row+1,:])[0].shape[0] >0:
        #i = np.nonzero(dfclpw.iloc[row,:])[0][0] #donde estoy
        i = np.nonzero(dfclpw.iloc[row,:])[0][0] #donde estoy
        j = np.nonzero(dfclpw.iloc[row+1,:])[0][0] #siguiente linea ( o estado)
        mattra[i,j]=mattra[i,j] +1


ValueError: Length of values (1) does not match length of index (25)

In [ ]:
columnas

In [ ]:
#convertir de numpy array to dataframe
idxclnames ={}
clidx = range(0,n_clusters)
clnames = []
for i in range(n_clusters):
    clnames.append('C'+str(i+1))

for i in clidx:
        idxclnames[i] = clnames[i]
    
dfMaTra =pd.DataFrame(mattra)
dfMaTra.columns=columnas
dfMaTra.rename(index=idxclnames)


In [ ]:
dfMaTra.C19

In [ ]:
#la funcion se aplica fila por fila
#se suman todos los valores por fila y se divide entre cada valor, luego se multiplica por 100
def calculate_vectorprob(row):
    return row.astype(float)/row.sum()*100

mprob = dfMaTra.apply(calculate_vectorprob, axis=1)
mprob.rename(index=idxclnames,inplace=True)


In [ ]:
#SOLO FUNCIONA PARA FILAS PERO NO PARA COLUMNAS
plt.figure(figsize=(15,15))

sns.heatmap(mprob, annot=True)
